In [2]:
import csv
import os
import requests as req
import json
from collections import Counter

In [3]:
#Donorschoose.org API
api_key = "DONORSCHOOSE"
url = "https://api.donorschoose.org/common/json_feed.html?max=100"

In [4]:
#Open a CSV for a list of Bay Area cities
with open("bayarea_cities.csv") as cities:
    cities_reader = csv.reader(cities)
    
    cities = [city[0] for city in cities_reader]
    
    print(len(cities))

cities = cities[1:6]
print(cities)

102
['Alameda', 'Albany', 'American Canyon', 'Antioch', 'Atherton']


In [5]:
# Build partial query URL
query_url = url + "&APIkey=" + api_key + "&state=CA" + "&cityName="
#location_data = [req.get(query_url + city).json() for city in cities]

all_data = {}

for city in cities:
    
    results = req.get(query_url + city).json()
    all_data[city] = results   

print(json.dumps(all_data, indent=2))

{
  "Alameda": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Alameda&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "8",
    "index": "0",
    "max": "8",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": [
      {
        "id": "2770068",
        "hashCode": "-1394581691",
        "proposalURL": "https://www.donorschoose.org/project/chromebooks-for-math/2770068/?verify=-1394581691&utm_source=api&utm_medium=feed&utm_content=bodylink&utm_campaign=DONORSCHOOSE",
        "fundURL": "https://secure.donorschoose.org/donors/givingCart.html?proposalid=2770068&donationAmount=45&verify=-1394581691&utm_source=api&utm_medium=feed&utm_content=fundlink&utm_campaign=DONORSCHOOSE",
        "imageURL": "https://storage.donorschoose.net/dc_prod/

In [6]:
city_data = {}

for cityname, data in all_data.items():
        
    #step 1: get cost to complete
    #step 2: get povertyType label
    #step 3: add this to the output vairable
    
    costcomplete = []
    povertytype = []
    
    data["proposals"]
    
    for x in data["proposals"]:
        
        costcomplete.append(float(x["costToComplete"]))
        povertytype.append(x["povertyType"]["label"])
    
    city_data[cityname] = {
        
        "costToComplete": sum(costcomplete),
        "povertyType": Counter(povertytype)
    }

city_data

{'Alameda': {'costToComplete': 12302.28,
  'povertyType': Counter({'LOW': 2,
           'MODERATE': 2,
           'MODERATEHIGH': 2,
           'MODERATELOW': 2})},
 'Albany': {'costToComplete': 3385.08, 'povertyType': Counter({'LOW': 3})},
 'American Canyon': {'costToComplete': 0, 'povertyType': Counter()},
 'Antioch': {'costToComplete': 27139.949999999997,
  'povertyType': Counter({'HIGH': 2,
           'HIGHEST': 8,
           'MODERATE': 5,
           'MODERATEHIGH': 19})},
 'Atherton': {'costToComplete': 0, 'povertyType': Counter()}}

In [7]:
#list of cityname, costtocomplete, number of low, number of modeatelow, etc.

csv_data = []

citynames = sorted(city_data.keys())

for cityname in citynames: 
    
    data = city_data[cityname]
    
    one_row = []
    
    one_row.append(cityname)
    one_row.append(data['costToComplete'])  
    
    labeltype = ["LOW", "MODERATELOW", "MODERATE", "MODERATEHIGH", "HIGH", "HIGHEST", "UNKNOWN"]
    
    for x in labeltype:
        
        one_row.append(data['povertyType'].get(x, 0))
    
    csv_data.append(one_row)

csv_data

[['Alameda', 12302.28, 2, 2, 2, 2, 0, 0, 0],
 ['Albany', 3385.08, 3, 0, 0, 0, 0, 0, 0],
 ['American Canyon', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Antioch', 27139.949999999997, 0, 0, 5, 19, 2, 8, 0],
 ['Atherton', 0, 0, 0, 0, 0, 0, 0, 0]]

In [12]:
root_path = os.path.join(os.getcwd())
data_path = os.path.join(root_path)
csvpath = os.path.join(root_path, "donorschoose_api.csv")
with open(csvpath, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    csvwriter.writerow(["City Name", "Cost To Complete", "LOW Poverty", "MODERATE-LOW Poverty", "MODERATE Poverty","MODERATE-HIGH Poverty","HIGH Poverty","HIGHEST Poverty", "Unknown"])
    csvwriter.writerows(csv_data)